In [1]:
import sqlite3

# Conecta ao banco de dados
conn = sqlite3.connect('academia.db')
cursor = conn.cursor()

# Cria tabelas (limpa se já existirem)
cursor.execute('DROP TABLE IF EXISTS clientes')
cursor.execute('DROP TABLE IF EXISTS mensalidades')

cursor.execute('''
CREATE TABLE clientes (
    id_cliente INTEGER PRIMARY KEY AUTOINCREMENT,
    nome TEXT NOT NULL,
    email TEXT UNIQUE NOT NULL
);
''')

cursor.execute('''
CREATE TABLE mensalidades (
    id_mensalidade INTEGER PRIMARY KEY AUTOINCREMENT,
    id_cliente INTEGER NOT NULL,
    valor REAL NOT NULL,
    data_pagamento TEXT NOT NULL,
    FOREIGN KEY (id_cliente) REFERENCES clientes(id_cliente)
);
''')

# Insere dados
clientes = [
    ('João Silva', 'joao@email.com'),
    ('Maria Souza', 'maria@email.com')
]
cursor.executemany("INSERT INTO clientes (nome, email) VALUES (?, ?)", clientes)

mensalidades = [
    (1, 100.0, '2025-05-01'),
    (2, 120.0, '2025-05-02')
]
cursor.executemany("INSERT INTO mensalidades (id_cliente, valor, data_pagamento) VALUES (?, ?, ?)", mensalidades)

conn.commit()

# Consultas

print("\n Total pago por cliente:")
cursor.execute('''
SELECT clientes.nome, SUM(mensalidades.valor) AS total_pago
FROM clientes
JOIN mensalidades ON clientes.id_cliente = mensalidades.id_cliente
GROUP BY clientes.id_cliente;
''')
for row in cursor.fetchall():
    print(row)

print("\n Clientes com mensalidades acima de R$100:")
cursor.execute('''
SELECT clientes.nome, mensalidades.valor
FROM clientes
JOIN mensalidades ON clientes.id_cliente = mensalidades.id_cliente
WHERE mensalidades.valor > 100;
''')
for row in cursor.fetchall():
    print(row)

print("\n Todas as mensalidades registradas:")
cursor.execute('''
SELECT clientes.nome, mensalidades.valor
FROM clientes
JOIN mensalidades ON clientes.id_cliente = mensalidades.id_cliente;
''')
for row in cursor.fetchall():
    print(row)

# Estatísticas
cursor.execute('SELECT SUM(valor) FROM mensalidades')
print(" Total pago:", cursor.fetchone()[0])

cursor.execute('SELECT AVG(valor) FROM mensalidades')
print(" Média das mensalidades:", cursor.fetchone()[0])

cursor.execute('SELECT MAX(valor) FROM mensalidades')
print(" Maior mensalidade:", cursor.fetchone()[0])

cursor.execute('SELECT MIN(valor) FROM mensalidades')
print(" Menor mensalidade:", cursor.fetchone()[0])

# Encerra
conn.close()
print("\nConexão encerrada.")



 Total pago por cliente:
('João Silva', 100.0)
('Maria Souza', 120.0)

 Clientes com mensalidades acima de R$100:
('Maria Souza', 120.0)

 Todas as mensalidades registradas:
('João Silva', 100.0)
('Maria Souza', 120.0)
 Total pago: 220.0
 Média das mensalidades: 110.0
 Maior mensalidade: 120.0
 Menor mensalidade: 100.0

Conexão encerrada.
